## testType_A data 탐색

In [3]:
data = read.csv("C://Users//이명아//Desktop//testType_A.csv")
head(data)
nrow(data)

monthCode,serviceId,userId,timeSpent,numberOfDays
201112,13,59749,136,3
201112,14,15220,33,1
201112,14,15471,812,19
201112,14,15583,38,1
201112,14,15777,6,1
201112,14,15792,1336,20


[1] 708844

In [ ]:
#결측치 확인
is.na(data)

#outlier 확인
a<-boxplot(data$serviceId)

b<-boxplot(data$timeSpent)
b$stats
data$timeSpent <- ifelse(data$timeSpent > 1360, NA , data$timeSpent)
table(is.na(data$timeSpent))

c<-boxplot(data$numberOfDays)
c$stats
data$numberOfDays<-ifelse(data$numberOfDays > 18, NA , data$numberOfDays)
table(is.na(data$numberOfDays))

d<-boxplot(data$userId)

In [ ]:
# outlier 제거
library(dplyr)
new_data <- data %>% filter(!is.na(timeSpent))
new_data <- new_data %>% filter(!is.na(numberOfDays))
nrow(new_data)   
#data: 708844 -> 604127로 감소

In [ ]:
# 1) 월별 서비스 이용 상황
# -1) 월별 timespent 합
ma1 <- tapply(new_data$timeSpent, new_data$monthCode, sum)
ma1
ma2$a<-factor(new_data$monthCode)

a<-factor(new_data$monthCode)

new_data$monthCode <- as.character(new_data$monthCode)
new_data[,month:=substr(monthCode,1,4)]

new_data2 <- new_data %>% mutate(month=substr(monthCode,5,6))

windows()

options(scipen = 999)

new_data2 %>% group_by(serviceId) %>% summarise(sum=sum(timeSpent)) %>%
  ggplot(aes(x=as.factor(serviceId),y=sum,group=1 ))+geom_line()
library(ggplot2)

windows()
  
serviceId <- as.factor(new_data$serviceId)
windows()
new_data2 %>% group_by(month,serviceId) %>% summarise(sum=sum(timeSpent)) %>%
  ggplot(aes(x=month,y=sum,group=serviceId,colour=serviceId))+geom_line()

ggplot(data = ma2) +
  aes(x = as.factor('new_data$monthCode'), y = `sum(timeSpent)`,group=1) +
  geom_line(color = "#0c4c8a") +
  theme_minimal()

# -2) 월별 numberOfDays 합

ma3 <- tapply(new_data$numberOfDays , new_data$monthCode, sum)
ma3

ma4 <- as.data.frame(summarise(group_by(new_data, new_data$monthCode), sum(numberOfDays)))
esquisse::esquisser(data = ma4)


ggplot(data = ma4) +
  aes(x = `new_data$monthCode`, y = `sum(numberOfDays)`) +
  geom_line(color = "#0c4c8a") +
  theme_minimal()

# 2) servicedID 
# -1) serviceId 별 timespent 평균
ma5 <- as.data.frame(summarise(group_by(new_data,  new_data$serviceId), mean= mean(timeSpent)))
esquisse::esquisser(data = ma5)

ggplot(data = ma5) +
  aes(x = `new_data$serviceId`, y = mean) +
  geom_line(color = "#0c4c8a") +
  theme_minimal()

# -2) serviceId 별 numberOfDays 평균
ma6 <- as.data.frame(summarise(group_by(new_data,  new_data$serviceId), mean= mean(numberOfDays)))
windows()
esquisse::esquisser(data = ma6)


ggplot(data = ma6) +
  aes(x = `new_data$serviceId`, y = mean) +
  geom_line(color = "#0c4c8a") +
  theme_minimal()
head(new_data)

# -4) monthCode별 servicId 별 timespent 합

k <-as.data.frame(summarise(group_by(new_data, new_data$monthCode, new_data$serviceId), sum(timeSpent)))
esquisse::esquisser(data = k)

### xgboost format

In [ ]:
set.seed(1234)
train_idx <- sample(nrow(new_data), floor(0.7*nrow(new_data)), replace =F)
train_X <- new_data[train_idx, 4:5]
train_y <- new_data[train_idx, 2]
test_X <- new_data[-train_idx, 4:5]
test_y <- new_data[-train_idx, 2]

library(xgboost)

dtrain <- xgb.DMatrix(data = data.matrix(train_X ) , label = train_y)
dtest <- xgb.DMatrix(data = data.matrix(test_X ) , label = test_y)

params <- list(booster = 'gbtree' , objective = 'multi:softmax',
               eval.metric = 'logloss', eta = 0.01, gamma = 0.04,
               max_depth = 6, scale_pos_weight = 1.0506,
               min_child_weight = 1, subsample = 0.732,
               colsample_bytree = 0.4698)

xgb_model <- xgb.train(params = params ,
                       data = dtrain,
                       nrounds = 7000,
                       watchlist = list(train = dtrain, eval = dtest),
                       print_every_n = 10,
                       early_stopping_rounds = 100,
                       maximize = F)